<img src="images/logodwengo.png" alt="logodwengo" style="width:200px;"/>

<div style='color: #690027;' markdown="1">
<h1>FROM LEAF TO LABEL: STOMATA DETECTION</h1></div>

<div class="alert alert-box alert-success">
In this notebook you train and test your own deep neural network to detect stomata. The methodology is the same as explained in the paper by Meeus et al. [1].</div>

<img src="images/stomatamethodologie.png" alt="methodologie" style="width:600px;"/>

As illustrated above, a window slides over your microphotograph (A). This *sliding window* thus divides your photo into small overlapping areas or patches (B) of 120 by 120 pixels. A deep neural network (VGG19) has been trained to label these patches (C). Positively labeled patches from a microphotograph are clustered (D), resulting in detection (E). This detection depends on the threshold value, the *threshold*, that you chose.

### Import necessary modules
You start by loading some Python modules:
- [PIL](https://pillow.readthedocs.io/en/stable/): a useful Python module for working with images;- [NumPy](https://numpy.org): the basic module to perform scientific operations in Python;- [sklearn](https://scikit-learn.org/stable/): the scikit-learn module for machine learning, particularly for the functionality of clustering;- [os](https://docs.python.org/3/library/os.html): a Python module for functionalities that depend on the operating system, e.g. reading, writing, and listing files;- [Matplotlib](https://matplotlib.org): a Python module for creating graphs.
A deep neural network consists of multiple layers that are linked together. The Python module Keras provides building blocks to build a neural network. The necessary functionalities are included in the underlying code. For calculations with tensors and other mathematical operations, Keras itself relies on the TensorFlow platform.

In [ ]:
from PIL import Image, ImageEnhance, ImageOpsimport numpy as npfrom sklearn.cluster import MeanShift, estimate_bandwidthimport osimport matplotlib.pyplot as plt

Next, you load multiple [Keras](https://keras.io/getting_started/intro_to_keras_for_researchers/) modules.

In [ ]:
from tensorflow.keras import backend as Kfrom tensorflow.keras.layers import Input, Convolution2D, Conv2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Densefrom tensorflow.keras.models import Model, load_modelfrom tensorflow.keras.utils import get_source_inputsfrom tensorflow.keras.utils import get_filefrom tensorflow.python.keras.utils import layer_utilsimport tensorflow.keras as kerasfrom tensorflow.keras.preprocessing.image import load_imgfrom tensorflow.keras.preprocessing.image import img_to_arrayfrom tensorflow.keras.preprocessing.image import ImageDataGeneratorimport tensorflow as tf
# limit GPU VRAMconfig = tf.compat.v1.ConfigProto()config.gpu_options.allow_growth = Truesess = tf.compat.v1.Session(config=config)

<div style='color: #690027;' markdown="1">
    <h2>1. Dataset</h2></div>

To train a *deep learning* model, you need data. As mentioned earlier, the deep learning model will detect stomata on square patches of 120 by 120 pixels. This is because it is trained with such patches. To obtain a robust model, both positive and negative examples must be presented to the system. Positive examples are examples with a stoma, negative examples are examples without a stoma.
The data is divided into three parts:- The training set, these are the data that are used to adjust the weights, the *weights*, of the (deep) neural network;- The validation set, these are the data used to see how well the learning process is progressing and to fine-tune the hyperparameters of the model;- The test set, these are the data you give to the system after training to test the developed model.
This notebook contains the training and validation of the *deep learning* system for stomata detection, and a small dataset limited to *Carapa procera* suitable for educational purposes. As a result, the computational needs have also been kept within reasonable limits (a full training with multiple plant species (see the paper) requires more time and patience).
After the training, the model should ideally be tested on the test set. In this notebook, this test is limited to one image. This is due to time constraints and because this is sufficient for the purpose of this notebook: to demonstrate how a convolutional neural network for stomata detection is built, trained, validated, and finally deployed.

First, download and unzip the dataset:

In [ ]:
!wget https://zenodo.org/record/3902280/files/data.zip!unzip "data.zip"

In [ ]:
train_dir = "./data/training/"val_dir = "./data/validation/"

The training and validation data contain patches of 120 by 120 pixels. A positively labeled patch shows a stoma:
<img src="images/carapapositief.jpg" width="120" />
    
A negatively labeled patch of *Carapa procera* has no stoma (unless perhaps a part of it):
<img src="images/carapanegatief.jpg" width="120" />

To obtain such patches, you need to have annotated micrographs (micrographs of which you know the coordinate of the center of the present stoma). The patches can then, based on these coordinates, be cut out using the [*crop*-function](https://pillow.readthedocs.io/en/stable/reference/Image.html) of PIL or even simpler by using [*matrix slicing*](https://numpy.org/doc/stable/reference/arrays.indexing.html) in NumPy.

The number of elements in the dataset is increased through *data augmentation*. The preprocessor [ImageDataGenerator](https://keras.io/api/preprocessing/image/#imagedatagenerator-class), defines the *data augmentation* that will be applied to the dataset. Here, it consists of random rotations, and horizontal and vertical *flips* of the patches.

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=180, horizontal_flip=True, vertical_flip=True, rescale=1/255.)

Besides determining the *data augmentation* that will be applied, you also use the ImageDataGenerator to capture some things:- the dimensions of the patches (120 x 120 pixels);- the color mode (grayscale or rgb);- the batch size (this is the number of samples that are used in an epoch of training, so in each training iteration);- the classification type of the task to be accomplished (here binary classification: a patch gets either a positive or a negative label);- whether the data should be shuffled or not;- the *seed*, the starting point, of the random number generator.
Finally, you provide a path to the directory with the training data.

In [ ]:
batch_size = 128

In [ ]:
train_generator = train_datagen.flow_from_directory(    directory=r"./data/training/",    target_size=(120, 120),    color_mode="rgb",    batch_size=batch_size,    class_mode="binary",    shuffle=True,seed=53The assistant wouldn't be able to perform the task without the text. Please provide the text you want to be translated.

For the validation as well, you configure an ImageDataGenerator. You define this one with the same properties as the one for training but without *data augmentation*.

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255.)
validation_generator = test_datagen.flow_from_directory(r"./data/validation/",        target_size=(120, 120),        color_mode="rgb",batch_size=batch_size,class_mode='binary')

<div style='color: #690027;' markdown="1">
<h2>2. Network architecture with necessary parameters</h2></div>

You start from the convolutional neural network of the [VGG19 model](https://arxiv.org/abs/1409.1556) to which you add two *dense layers*. <br>The convolutional neural layers have been pre-trained on [ImageNet](https://ieeexplore.ieee.org/abstract/document/5206848). Consequently, only the *dense layers* still need to be trained.<br>You download the pre-trained weights of the convnet from Keras using the keyword 'imagenet'.

In [ ]:
number_dense_neurons = 2048

In [ ]:
# We start from VGG19from tensorflow.keras.applications import VGG19
# We start with the convolutional layers of VGG19 with pre-trained weights on Imagenetvgg19_base = VGG19(weights="imagenet",include_top=False,input_shape=(120,120,3))x = vgg19_base.outputx = Flatten()(x)
# We add our own classification layersx = Dense(2*number_dense_neurons,activation='relu')(x)x = Dropout(0.5)(x)x = Dense(number_dense_neurons,activation='relu')(x)x = Dropout(0.5)(x)
# We add an output layerx = Dense(1,activation="sigmoid")(x)
model = Model(inputs=vgg19_base.input, outputs=x)
# We set that we do not train the (pre-trained) VGG19 layersfor layer in vgg19_base.layers:    layer.trainable = False
# What does the network look likemodel.summary()

<div style='color: #690027;' markdown="1">
<h2>3. Train the model and save it</h2></div>

The parameters are optimized by calling on the optimization function [Adam](https://arxiv.org/pdf/1412.6980.pdf); for this, the *learning rate* was fine-tuned and eventually set at 0.000005. Finally, you record the *loss* and the *metrics* for training and validation.

In [ ]:
learning_rate = 0.000005# Initialize Stochastic Gradient Descent with momentum, learning rate for fine-tuningopt = tf.keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)# Determine the loss and metrics for training and validationmodel.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy"])

Training is done through the function *fit()* for 50 epochs. Note that the architecture has been configured in such a way that only the weights of the *dense layers* are adjusted.

In [ ]:
epochs = 50The input provided does not contain any Dutch text that needs to be translated.        train_generator,        epochs=epochs,        validation_data=validation_generator)

Now you are ready for the next step. After all, the network has been trained and can now be used.<br>To use the AI system, you need to save the model parameters. You can do this by following the instruction *model.save(path)* with *path* being the path to the file in which you want to save the parameters.<br>Moreover, the function *fit()* returns a *history* object. This object includes the progress of the training and validation across the different epochs. Consequently, this is useful for monitoring the training process, for example to compare the results of different settings of the hyperparameters, such as the *learning rate*, the number of *epochs*, and the size of the *batches*.

In [ ]:
# Save the Carapa procera deep learning modelmodel.save("my_carapa_procera_model")
# Display the performance for training and validationplt.plot(history.history["loss"], label="Training loss")plt.plot(history.history["val_loss"], label="Validation loss")

<div style='color: #690027;' markdown="1">
<h2>4. Load the deep learning model</h2></div>

Now you have trained a first deep learning model for stomata detection in the *Carapa procera*. This model is saved as the object *model*.<br> If you want to start from a saved deep learning model, you retrieve it from the file by executing the instruction *model = load_model(path_to_model)*.

In [ ]:
# Uncomment the following line if you want to work with your previously saved model without going through the training process again# model = load_model("my_carapa_procera_model")

<div style='color: #690027;' markdown="1">
<h2>5. Image and detection parameters</h2></div>

The model can only detect stomata on images of 120 by 120 pixels. Therefore, an offered image must first be divided into patches. The model uses a method with a *sliding window* for this.<br>Although this method is not the most (computationally) efficient, it is very easy to understand. The window is 120 by 120 pixels big and shifts each time by a step of 10 pixels.<br> You start by loading your image:

In [ ]:
demo_image = "./data/Carapa_procero_demo.jpg" # You can use a different Carapa procero microphoto

In [ ]:
image = Image.open(demo_image)fig, ax = plt.subplots(figsize=(20, 10))image = np.array(image) # Conversion to Numpy arrayax.imshow(image)

In [ ]:
shift = 10patch_size = 120

The number of slides that are executed is also part of the detection parameters:

In [ ]:
no_x_shifts = (np.shape(image)[0] - patch_size) // shiftno_y_shifts = (np.shape(image)[1] - patch_size) // shiftprint("We do "+str(no_x_shifts*no_y_shifts)+" shifts. Consequently, the deep learning model is applied to "+str(no_x_shifts*no_y_shifts)+" patches.")

<div style='color: #690027;' markdown="1">
<h2>6. Classification with the deep learning model</h2></div>

Now that all windows have been identified, the deep learning model can take action. You accomplish this by calling the *predict()* function. However, first you have to convert the used image (change it to the expected format) and normalize it (elements get values from 0 to 1).<br>The output of the deep learning model is a number between 0 and 1 that indicates how confident the model is that the image shows a stoma. Therefore, you also need to establish a threshold value, *threshold*, from which the output is accepted as a positive classification. The higher this threshold, the stricter the system will act in detecting the stomata. However, if the threshold is too high, the system will not be able to detect a single stoma. The threshold below is the same as in the paper:

In [ ]:
threshold = 0.7

In [ ]:
patches = []coordinates = []stomata = []offset = patch_size // 2for x in np.arange(no_x_shifts + 1):    for y in np.arange(no_y_shifts + 1):        # Middle of the window        x_c = x * shift + offset        y_c = y * shift + offset
# Cutting out the window and converting to expected format before applying the deep learning model        patch = image[x_c - offset:x_c + offset, y_c - offset:y_c + offset, :]patch = patch.astype("float32")patch /= 255        
# Add window to the list of windowspatches.append(np.expand_dims(patch, axis=0))        
# Adding coordinates of the center of the window to the list of coordinatescoordinates.append([x_c, y_c])
# Process all windows in batches of 128 windows to perform calculations in parallelbatch_size = 128for b in range(0, len(patches), batch_size):batch = patches[b:b + batch_size]
# Applying the model to do the detection    y_model = model.predict(np.vstack(batch))
    for p in range(0, len(y_model)):# Stoma if the output of the model is above the threshold        if y_model[p] > threshold:stomata.append(coordinates[b + p])

<div style='color: #690027;' markdown="1">
<h2>7. Clustering of the detected stomata</h2></div>

All positively labeled patches are clustered using *mean shift clustering*. This technique groups neighboring (or even overlapping) positively labeled patches from which the coordinate of the effective stoma is derived. For this, you can use the [MeanShift](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html) module, which is available in [scikit-learn](https://scikit-learn.org).

In [ ]:
bandwidth = patch_size // 2
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)ms.fit(stomata)stomata = np.array([[x[1], x[0]] for x in ms.cluster_centers_]) # cluster_centers_ is inverted

<div style='color: #690027;' markdown="1">
<h2>8. Graphical representation of the results</h2></div>

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))ax.imshow(image)ax.plot(stomata[:,0], stomata[:,1], 'xr', alpha=0.75, markeredgewidth=3, markersize=12)

### Reference List

[1] Meeus, S., Van den Bulcke, J., & wyffels, F. (2020). From leaf to label: A robust automated workflow for stomata detection. *Ecology and evolution 10*(17),<br>&nbsp; &nbsp; &nbsp; &nbsp; 9178-9191. [doi:10.1002/ece3.6571](https://doi.org/10.1002/ece3.6571) <br>[2] Simonyan, K., & Zisserman, A. (2014). Very deep convolutional networks for large-scale image recognition. *arXiv preprint*. [arXiv:1409.1556](https://arxiv.org/abs/1409.1556) <br>[3] Deng, J., et al. (2009). Imagenet: A large-scale hierarchical image database. *IEEE conference on computer vision and pattern recognition*. [IEEE](https://ieeexplore.ieee.org/abstract/document/5206848) <br>[4] Kingma, D. P., & Ba, J. (2014). Adam: A method for stochastic optimization. *arXiv preprint*. [arXiv:1412.6980](https://arxiv.org/pdf/1412.6980.pdf)

<img src="images/cclic.png" alt="Banner" align="left" style="width:100px;"/><br><br>
Notebook KIKS, see <a href="http://www.aiopschool.be">AI at School</a>, by F. wyffels for Dwengo vzw, licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.